In [1]:
import numpy as np
import h5py
import torch 
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torchvision.transforms as transforms

In [2]:
with h5py.File('/kaggle/input/electron-photon-dataset/Electron.hdf5','r') as h5:
    ElectronX = h5['X'][:]
    ElectronY = h5['y'][:]

with h5py.File('/kaggle/input/electron-photon-dataset/Photon.hdf5','r') as h5:
    PhotonX = h5['X'][:]
    PhotonY = h5['y'][:]

ElectronX.shape, PhotonX.shape, ElectronY.shape, PhotonY.shape

((249000, 32, 32, 2), (249000, 32, 32, 2), (249000,), (249000,))

In [3]:
X = np.concatenate([ElectronX, PhotonX])
Y = np.concatenate([ElectronY, PhotonY])

indexes = np.random.permutation(len(X))
X = X[indexes]
Y = Y[indexes]

x_train, x_test = np.split(X, [int(0.8*len(X))])
y_train, y_test = np.split(Y, [int(0.8*len(Y))])

In [4]:
np.sum(y_train == 0) / len(y_train), np.sum(y_test == 0) / len(y_test) 

(0.5004241967871486, 0.4983032128514056)

In [5]:
class CustomDataset(Dataset):
    def __init__(self, X, Y, transform=None, label_transform = None):
        self.X = X
        self.labels = Y
        self.transform = transform
        self.label_transform = label_transform


    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        sample = self.X[idx]
        label = self.labels[idx]
        
        if self.transform:
            sample = (self.transform(sample))

        if self.label_transform:
            label = np.array(label)
            label = self.label_transform(label)

        return sample, label

CustomTransform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5], [0.5, 0.5])
])

def OneHotEncoder(label):
    return torch.nn.functional.one_hot(label, 2)

trainDS = CustomDataset(x_train, y_train, CustomTransform)
testDS = CustomDataset(x_test, y_test, CustomTransform)

trainDL = DataLoader(trainDS, batch_size=32, shuffle=True)
testDL =  DataLoader(testDS,  batch_size=32, shuffle=True)

In [6]:
class ConvLayerStack(nn.Module):

    def __init__(self, in_c, out_c, kernel, stride, padding):
        super().__init__()
        self.Conv1 = nn.Conv2d(in_c, in_c//4, kernel, stride, padding)
        # self.Conv2 = nn.Conv2D(in_c/4, in_c/4, kernel, stride, padding)
        self.Conv3 = nn.Conv2d(in_c//4, out_c, kernel, stride, padding)
        self.BNorm1 = nn.BatchNorm2d(in_c//4)
        self.BNorm2 = nn.BatchNorm2d(out_c)
        self.Relu = nn.ReLU()

    def forward(self, x):
        f = self.Conv3( self.Relu( self.BNorm1(self.Conv1(x)) ))
        # print(f.shape)
        return self.Relu(self.BNorm2(f + x))


class ResNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.Conv1 = nn.Conv2d(2,64,2,2,0)
        self.Pooling1 = nn.AvgPool2d(1,1)
        self.Res1 = ConvLayerStack(64,64,3,1,1)
        self.Res2 = ConvLayerStack(64,64,3,1,1)
        self.Res3 = ConvLayerStack(64,64,3,1,1)
        self.Res4 = ConvLayerStack(64,64,3,1,1)
        self.Res5 = ConvLayerStack(64,64,3,1,1)
        self.Res6 = ConvLayerStack(64,64,3,1,1)
        self.Pooling2 = nn.AvgPool2d(2,2,0)
        self.Lin = nn.Linear(64*8*8, 2)

    def forward(self,x):
        # print(x.shape)
        f = self.Conv1(x)
        # print(f.shape)
        f = self.Pooling1(f)
        # print(f.shape)
        f = self.Res1(f)
        f = self.Res2(f)
        f = self.Res3(f)
        f = self.Res4(f)
        f = self.Res5(f)
        f = self.Res6(f)
        # print(f.shape)
        f = self.Pooling2(f)
        # print(f.shape)
        f = torch.flatten(f,1)
        # print(f.shape)
        return self.Lin(f)
        

In [7]:
from tqdm import tqdm
import torch
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F

def Train(model, data, opti, loss_fn):
    model.train()
    Dsize = len(data.dataset)
    NBatches = len(data)

    loss = 0.
    correct = 0
    
    for (x,y) in tqdm(data, total=NBatches):
        x, y = x.to("cuda"), y.to("cuda")
        pred = model(x)
        loss_current = loss_fn(pred, y.long())

        opti.zero_grad()
        loss_current.backward()
        opti.step()
        opti.zero_grad()

        loss += loss_current.item()
        correct += (pred.argmax(1) == y.long()).type(torch.float).sum().item()

    print(f"Training Error: {1 - (correct/Dsize)}, Loss: {loss/NBatches}")

def Test(model, dataloader, criterion, device='cuda'):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    total_correct = 0
    total_samples = 0
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)  # shape: (batch_size, 2)
            loss = criterion(outputs, labels.long())
            
            # Accumulate loss
            total_loss += loss.item() * inputs.size(0)
            
            # Predictions and accuracy
            preds = torch.argmax(outputs, dim=1)  # Get predicted class
            total_correct += (preds == labels.long()).sum().item()
            total_samples += labels.size(0)
            
            # Compute probabilities for AUC
            probs = F.softmax(outputs, dim=1)[:, 1]  # Probability of class 1
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute average loss
    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples

    # Compute AUC score
    auc_score = roc_auc_score(all_labels, all_probs)

    print(f"Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}, AUC: {auc_score:.4f}")
    return avg_loss, accuracy, auc_score
        

In [8]:
ResNN = ResNet().to("cuda")

loss_fn = nn.CrossEntropyLoss()
opti = torch.optim.RMSprop(ResNN.parameters(), lr=10e-5)

In [9]:
print(ResNN)

ResNet(
  (Conv1): Conv2d(2, 64, kernel_size=(2, 2), stride=(2, 2))
  (Pooling1): AvgPool2d(kernel_size=1, stride=1, padding=0)
  (Res1): ConvLayerStack(
    (Conv1): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Conv3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (BNorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (BNorm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Relu): ReLU()
  )
  (Res2): ConvLayerStack(
    (Conv1): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Conv3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (BNorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (BNorm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Relu): ReLU()
  )
  (Res3): ConvLayerStack(
    (Conv1): Conv2d(64, 16, kernel_size=(3, 3), stride=(

In [10]:
epochs = 15
max_acc = 0

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    Train(ResNN, trainDL, opti, loss_fn,)
    a, cur_acc, b = Test(ResNN, testDL, loss_fn)
    if cur_acc >= max_acc:
        filename = "BestModel.pth"
        torch.save(ResNN, filename)
        max_acc = cur_acc

Epoch 1
-------------------------------


100%|██████████| 12450/12450 [02:12<00:00, 94.13it/s]


Training Error: 0.3709738955823293, Loss: 0.6443124474411509
Loss: 0.6192, Accuracy: 0.6614, AUC: 0.7158
Epoch 2
-------------------------------


100%|██████████| 12450/12450 [02:11<00:00, 94.60it/s]


Training Error: 0.32300451807228914, Loss: 0.605577564579416
Loss: 0.7364, Accuracy: 0.5316, AUC: 0.5997
Epoch 3
-------------------------------


...

Training Error: 0.27160391566265063, Loss: 0.5467158416667617
Loss: 0.7051, Accuracy: 0.5308, AUC: 0.5380
Epoch 14
-------------------------------


100%|██████████| 12450/12450 [02:15<00:00, 92.15it/s]


Training Error: 0.2696561244979919, Loss: 0.543950444981755
Loss: 0.6019, Accuracy: 0.7007, AUC: 0.7761
Epoch 15
-------------------------------


100%|██████████| 12450/12450 [02:13<00:00, 93.42it/s]


Training Error: 0.267464859437751, Loss: 0.5407315905529811
Loss: 0.6066, Accuracy: 0.6931, AUC: 0.7694


In [11]:
max_acc

0.715773092369478

In [12]:
opti = torch.optim.RMSprop(ResNN.parameters(), lr=10e-8)

In [13]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    Train(ResNN, trainDL, opti, loss_fn,)
    a, cur_acc, b = Test(ResNN, testDL, loss_fn)
    if cur_acc >= max_acc:
        filename = "BestModel.pth"
        torch.save(ResNN, filename)
        max_acc = cur_acc

Epoch 1
-------------------------------


100%|██████████| 12450/12450 [02:11<00:00, 94.77it/s]


Training Error: 0.2622038152610442, Loss: 0.5335396448076968
Loss: 0.5579, Accuracy: 0.7214, AUC: 0.7864
Epoch 2
-------------------------------


100%|██████████| 12450/12450 [02:13<00:00, 93.03it/s]


Training Error: 0.25990461847389557, Loss: 0.5301572766744468
Loss: 0.5560, Accuracy: 0.7229, AUC: 0.7878
Epoch 3
-------------------------------


...

Training Error: 0.25550451807228913, Loss: 0.523900556985633
Loss: 0.5543, Accuracy: 0.7238, AUC: 0.7899
Epoch 14
-------------------------------


100%|██████████| 12450/12450 [02:14<00:00, 92.49it/s]


Training Error: 0.25518574297188756, Loss: 0.5235524809863193
Loss: 0.5545, Accuracy: 0.7237, AUC: 0.7900
Epoch 15
-------------------------------


 31%|███       | 3854/12450 [00:42<01:30, 95.47it/s]

Loss: 0.5543, Accuracy: 0.7237, AUC: 0.7898


In [14]:
max_acc

0.7238855421686747

# Final result:
We obtain:
Best *Validation* accuracy: *0.7238*<br>
Best *Traning* accuracy: *0.7439*<br>
Best *AUC* Score is: *0.7887*<br>


In [15]:
opti = torch.optim.Adam(ResNN.parameters(), lr=10e-9)

In [16]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    Train(ResNN, trainDL, opti, loss_fn,)
    a, cur_acc, b = Test(ResNN, testDL, loss_fn)
    if cur_acc >= max_acc:
        filename = "BestModel.pth"
        torch.save(ResNN, filename)
        max_acc = cur_acc

Epoch 1
-------------------------------


100%|██████████| 12450/12450 [02:16<00:00, 91.41it/s]


Training Error: 0.2548744979919678, Loss: 0.5232296316307712
Loss: 0.5543, Accuracy: 0.7239, AUC: 0.7902
Epoch 2
-------------------------------


100%|██████████| 12450/12450 [02:16<00:00, 91.22it/s]


Training Error: 0.2552334337349398, Loss: 0.5232127262478372
Loss: 0.5542, Accuracy: 0.7238, AUC: 0.7899
Epoch 3
-------------------------------


...

Training Error: 0.25511797188755025, Loss: 0.5230412023709002
Loss: 0.5543, Accuracy: 0.7236, AUC: 0.7899
Epoch 14
-------------------------------


100%|██████████| 12450/12450 [02:19<00:00, 89.38it/s]


Training Error: 0.2545456827309237, Loss: 0.5229674203759695
Loss: 0.5543, Accuracy: 0.7239, AUC: 0.7899
Epoch 15
-------------------------------


100%|██████████| 12450/12450 [02:22<00:00, 87.34it/s]


Training Error: 0.2549548192771084, Loss: 0.5229130106159003
Loss: 0.5541, Accuracy: 0.7239, AUC: 0.7902


# Final Final results:

Best AUC Score: 0.7902
Best Accuracy Score (Validation) : 0.7244
